In [ ]:
!pip install -q git+https://github.com/huggingface/transformers
!pip install -qU langchain Faiss-gpu tiktoken sentence-transformers
!pip install -qU trl Py7zr auto-gptq optimum
!pip install -q rank_bm25
!pip install -q PyPdf

In [2]:
import langchain
from langchain.embeddings import CacheBackedEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.llms import HuggingFacePipeline
from langchain.cache import InMemoryCache
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import prompt
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain import PromptTemplate
#
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
dir_loader = DirectoryLoader("/content/pdf",
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
docs = dir_loader.load()
#
print(f"len of documents in :{len(docs)}")

len of documents in :65


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                      chunk_overlap=200,)
#
esops_documents = text_splitter.transform_documents(docs)
print(f"number of chunks in barbie documents : {len(esops_documents)}")

number of chunks in barbie documents : 972


In [ ]:
store = LocalFileStore("./cache/")
embed_model_id = 'BAAI/bge-small-en-v1.5'
core_embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id)
embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model,
                                                  store,
                                                  namespace=embed_model_id)
# Create VectorStore
vectorstore = FAISS.from_documents(esops_documents,embedder)

In [6]:
bm25_retriever = BM25Retriever.from_documents(esops_documents)
bm25_retriever.k=5

In [30]:
query = "What is direct scope1 GHG emission in metric tone?"
embedding_vector = core_embeddings_model.embed_query(query)
print(len(embedding_vector))
#
docs_resp = vectorstore.similarity_search_by_vector(embedding_vector,k=5)
#
for page in docs_resp:
  print(page.page_content)
  print("\n")

384
1 Scope 1 emissions are direct GHG emissions from sources we own or control, such as on-site fuel combustion. Scope 2 emissions are indirect GHG emissions associated with 
purchased electricity and steam for owned/controlled facilities. Baxter’s Scope 1 and Scope 2 emissions have been verified by a third party to a reasonable assurance level (see


Scope 1
Direct GHG emissions from 
sources we own or control 
including fuel combustionGovernance 
and Data
Scope 2
Indirect GHG emissions 
from purchased electricity
and steam
Scope 3
Indirect GHG emissions 
from our value chain2025 2030 2040GOAL: Achieve carbon neutralit y 
for our direct operations (Scope 
1 and 2 GHG emissions) by 2040GOAL: Reduce absolute 
Scope 1 and 2 GHG 
emissions by 25% by 2030
2020BaselineOur History
1997   Began reporting GHG emissions


for our direct operations (Scope 
1 and 2 GHG emissions) by 2040GOAL: Reduce absolute 
Scope 1 and 2 GHG 
emissions by 25% by 2030
2020BaselineOur History
1997   Began report

In [31]:
%%timeit -n 1 -r 1
query = "What is direct scope1 GHG emission in metric tone?"
#
embedding_vector = core_embeddings_model.embed_query(query)
docs_resp = vectorstore.similarity_search_by_vector(embedding_vector,k=5)

25.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [32]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k":5})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever,faiss_retriever],
                                       weights=[0.5,0.5])

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="gptq-8bit-32g-actorder_True")
#
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

KeyboardInterrupt: ignored

In [33]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [34]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [35]:
langchain.llm_cache = InMemoryCache()

In [36]:
PROMPT_TEMPLATE = '''
You are my ESG proffesional advisor. You are great at providing answer with your knowledge in ESG.
With the information being provided try to answer the question.
If you cant answer the question based on the information either say you cant find an answer or unable to find an answer.
So try to understand in depth about the context and answer only based on the information provided. Dont generate irrelevant answers

Context: {context}
Question: {question}
Do provide only helpful answers

Helpful answer:
'''
#
input_variables = ['context', 'question']
#
custom_prompt = PromptTemplate(template=PROMPT_TEMPLATE,
                            input_variables=input_variables)

In [37]:
handler = StdOutCallbackHandler()
#
qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = vectorstore.as_retriever(search_kwargs={"k":5}),
    verbose=True,
    callbacks=[handler],
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)

In [38]:
%%time
query = "What is direct scope1 GHG emission in metric tone?"
response = qa_with_sources_chain({"query":query})
print(f"Response generated : \n {response['result']}")
print(f"Source Documents : \n {response['source_documents']}")



> Entering new RetrievalQA chain...


KeyboardInterrupt: ignored

In [41]:
#
handler = StdOutCallbackHandler()
#
qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = ensemble_retriever,
    callbacks=[handler],
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)

In [42]:
%%time
query = "What is direct scope1 GHG emission in metric tone?"
response = qa_with_sources_chain({"query":query})
print(f"Response generated : \n {response['result']}")
print(f"Source Documents : \n {response['source_documents']}")



> Entering new RetrievalQA chain...

> Finished chain.
Response generated : 
 
The direct scope 1 GHG emissions for Baxter in 2020 were 13.6% reduction from a 2015 baseline. To calculate this, we need to know the total scope 1 GHG emissions in 2015 and then subtract the percentage reduction achieved by 2020. Unfortunately, the information provided does not include the total scope 1 GHG emissions in 2015. However, we can use the percentage reduction achieved by 2020 to estimate the total scope 1 GHG emissions in 2015.

Assuming that the total scope 1 GHG emissions in 2015 were equal to the total scope 1 GHG emissions in 2020, we can calculate the total scope 1 GHG emissions in 2015 as follows:

Total scope 1 GHG emissions in 2015 = Total scope 1 GHG emissions in 2020 x 1.136 / 0.864

Total scope 1 GHG emissions in 2015 = 13.6 x 1.136 / 0.864

Total scope 1 GHG emissions in 2015 = 16.4 metric tons CO2e

Therefore, the direct scope 1 GHG emissions for Baxter in 2020 were 13.6% reduction